# Installing Net2Brain

<img src="workshops/data/Net2Brain_Logo.png" width="25%" />

In [1]:
#!pip install -U git+https://github.com/cvai-roig-lab/Net2Brain

# Step 2: Creating Representational Dissimilarity Matrices (RDMs) using `RDM Creator`

After completing Step 1, where the Feature Extractor extracts features from the specified layers, we can now proceed to Step 2, which involves calculating Representational Dissimilarity Matrices (RDMs) using the RDM Creator's built-in functionality.

We first initialize the RDM Creator class, which contains the function to generate RDMs. The RDM Creator class requires the following parameters:
- **device**: The device on which the RDMs will be generated. This can be 'cpu' or 'cuda'.
- **verbose**: Whether to print the progress of the RDM generation process.

To generate RDMs, the RDM Creator function requires:

- **feature_path**: The path to the .npz files containing the layer features for each image. These files should have a [Batch x Channel x Height x Width] format.
- **save_path** (optional): The location where the generated RDMs will be saved.
- **save_format** (optional): The format in which the RDMs will be saved. This can be 'npz' or 'pt'.
- **distance** (optional): The distance function to generate the RDMs. Default is the correlation distance.
- **chunk_size** (optional): The number of images to be processed at a time. This is useful when the number of images is too large to fit in memory. Default is all images.



The function will then:

- **Output**: Create an RDM with the shape (#Images, #Images) for every specified layer.

In this example, we'll be using the AlexNet features generated earlier in the tutorial. The function can be called as follows:

In [ ]:
from net2brain.rdm_creation import RDMCreator

feat_path = "AlexNet_Feat"
save_path = "AlexNet_RDM"


# Call the Class with the path to the features
creator = RDMCreator(verbose=True, device='cpu') 
save_path = creator.create_rdms(feature_path=feat_path, save_path=save_path, save_format='npz') 

The default distance function is the correlation distance. To use a different distance function, we can specify the distance function in the **distance** parameter. The available distance functions can been seen by calling the **distance_functions** method of the RDMCreator. We created synonyms for the distance functions to make it easier to use them (i.e. l2 == euclidean). The available distance functions are:

In [ ]:
creator.distance_functions()

You can also use custom distance functions by passing a function to the **distance** parameter. The function should take one argument `x` of shape `(N, D)`, which represents the features (dimension `D`) of the `N` images and return a pairwise distance matrix of shape `(N, N)`. For example, we can use the cosine distance function as follows:

In [ ]:
import torch.nn.functional as F

def custom_cosine(x):
    x_norm = F.normalize(x, p=2, dim=1)
    return 1 - (x_norm @ x_norm.T)

creator.create_rdms(feature_path=feat_path, save_path='AlexNet_RDM_custom', save_format='npz', distance=custom_cosine)

We can access and visualize the RDM of a single layer using the `LayerRDM` class:

In [ ]:
from net2brain.rdm_creation import LayerRDM

rdm = LayerRDM.from_file("AlexNet_RDM/RDM_features_0.npz")
print(rdm)
rdm.plot(indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
from net2brain.rdm_creation import RDMCreator

feat_path = "ResNet50_Feat"
save_path = "ResNet50_RDM"


# Call the Class with the path to the feautures
creator = RDMCreator(device='cpu', verbose=True)  
creator.create_rdms(feature_path=feat_path, save_path=save_path, save_format='npz')  # This will create and save the RDMs at the specified save path